In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
%%time
data = pd.read_csv('Data/NY property data.csv')

Wall time: 13.5 s


In [3]:
data = data[['FULLVAL', 'AVLAND', 'AVTOT', 'LTFRONT', 'LTDEPTH', 'BLDFRONT', 'BLDDEPTH', 'ZIP', 'STORIES',
            'RECORD', 'B', 'BLOCK', 'LOT', 'TAXCLASS']]

In [4]:
data.replace(to_replace=0, value=np.nan, inplace = True)

In [5]:
data.head()

,FULLVAL,AVLAND,AVTOT,LTFRONT,LTDEPTH,BLDFRONT,BLDDEPTH,ZIP,STORIES,RECORD,B,BLOCK,LOT,TAXCLASS
0,21400000.0,4225500.0,9630000.0,500.0,1046.0,NaN,NaN,10004.0,NaN,1,1,1,101,4
1,193800000.0,14310000.0,87210000.0,27.0,NaN,NaN,NaN,10004.0,NaN,2,1,1,201,4
2,104686000.0,39008700.0,47108700.0,709.0,564.0,709.0,564.0,10004.0,3.0,3,1,2,1,4
3,39200000.0,15255000.0,17640000.0,793.0,551.0,85.0,551.0,10004.0,2.0,4,1,2,23,4
4,272300000.0,121050000.0,122535000.0,323.0,1260.0,89.0,57.0,10004.0,1.0,5,1,3,1,4


In [6]:
def clean_df():
    for key in data.keys():
        if '_y' in key:
            data.drop(columns=[key], inplace = True)
        elif key == 'key':
            data.drop(columns=[key], inplace = True)

In [7]:
def rename_col(df):
    key_dict = {}
    for key in df.keys():
        if key != 'key':
            key_dict[key] = key + '_y'
    return df.rename(columns = key_dict)

In [8]:
sum(data['ZIP'].isna())

29890

In [9]:
%%time
## Replacing missing values in ZIP column

groupby_count_med_ZIP = data.groupby(['B', 'BLOCK'])['ZIP'].agg(['count', pd.Series.mode]).reset_index()
groupby_count_med_ZIP['key'] = groupby_count_med_ZIP['B'].apply(str) + ' ' + groupby_count_med_ZIP['BLOCK'].apply(str)
data['key'] = data['B'].apply(str) + ' ' + data['BLOCK'].apply(str)
groupby_count_med_ZIP = rename_col(groupby_count_med_ZIP)
data = data.merge(groupby_count_med_ZIP, on = 'key', how = 'left', suffixes=('', '_y'))

def clean_zip(zip):
    try:
        return int(zip)
    except:
        try:
            return int(zip[0])
        except:
            return np.nan

data['mode_y'] = data['mode_y'].apply(clean_zip)
data['ZIP'].fillna(data['mode_y'], inplace = True)
clean_df()

zip_B = data.groupby('B')['ZIP'].agg(['count', pd.Series.mode])
zip_B = rename_col(zip_B)
data = data.merge(zip_B, on = 'B', how = 'left', suffixes=('', '_y'))
data['ZIP'].fillna(data['mode_y'], inplace = True)
clean_df()

Wall time: 26.4 s


In [10]:
sum(data['ZIP'].isna())

0

In [11]:
%%time
## Replacing missing values in STORIES column

stories_level1 = data.groupby(['B', 'TAXCLASS'])['STORIES'].agg(['count', 'median']).reset_index()
stories_level1 = stories_level1[stories_level1['count'] >= 5]
stories_level1['key'] = stories_level1['B'].apply(str) + ' ' + stories_level1['TAXCLASS']
data['key'] = data['B'].apply(str) + ' ' + data['TAXCLASS']
stories_level1 = rename_col(stories_level1)
data = data.merge(stories_level1, on = 'key', how = 'left', suffixes=('', '_y'))
data['STORIES'].fillna(data['median_y'], inplace = True)
clean_df()

stories_med = data.groupby('TAXCLASS')['STORIES'].agg(['count', 'median']).reset_index()
stories_med = rename_col(stories_med)
data = data.merge(stories_med, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['STORIES'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 7.97 s


In [12]:
%%time
# Replacing missing values in FULLVAL column

zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['FULLVAL'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['FULLVAL'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['FULLVAL'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['FULLVAL'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.36 s


In [13]:
%%time
## Replacing missing values in AVTOT column

zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['AVTOT'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['AVTOT'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['AVTOT'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['AVTOT'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.39 s


In [14]:
%%time
## Replacing missing values in AVLAND column

zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['AVLAND'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['AVLAND'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['AVLAND'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['AVLAND'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.36 s


In [15]:
%%time
## Replacing missing values in LTFRONT column

zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['LTFRONT'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['LTFRONT'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['LTFRONT'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['LTFRONT'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.4 s


In [16]:
%%time
## Replacing missing values in LTDEPTH column


zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['LTDEPTH'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['LTDEPTH'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['LTDEPTH'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['LTDEPTH'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.39 s


In [17]:
%%time
## Replacing missing values in BLDFRONT column

zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['BLDFRONT'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['BLDFRONT'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['BLDFRONT'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['BLDFRONT'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.42 s


In [18]:
%%time
## Replacing missing values in BLDDEPTH column

zip_tax = data.groupby(['TAXCLASS', 'ZIP'])['BLDDEPTH'].agg(['count', 'median']).reset_index()
zip_tax = zip_tax[zip_tax['count'] >= 5]
zip_tax['key'] = zip_tax['TAXCLASS'] + ' ' + zip_tax['ZIP'].apply(str)
zip_tax = rename_col(zip_tax)
data['key'] = data['TAXCLASS'] + ' ' + data['ZIP'].apply(str)
data = data.merge(zip_tax, on='key', how = 'left')
data['BLDDEPTH'].fillna(data['median_y'], inplace = True)
clean_df()

tax_val = data.groupby('TAXCLASS')['BLDDEPTH'].agg(['count', 'median']).reset_index()
tax_val = rename_col(tax_val)
data = data.merge(tax_val, left_on='TAXCLASS', right_on='TAXCLASS_y', how = 'left')
data['BLDDEPTH'].fillna(data['median_y'], inplace = True)
clean_df()

Wall time: 8.36 s


In [19]:
data.to_csv('Data/NY data no missing values.csv', index = False)

In [20]:
data.isna().sum()

FULLVAL     0
AVLAND      0
AVTOT       0
LTFRONT     0
LTDEPTH     0
BLDFRONT    0
BLDDEPTH    0
ZIP         0
STORIES     0
RECORD      0
B           0
BLOCK       0
LOT         0
TAXCLASS    0
dtype: int64